In [ ]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 1.2 MB/s 


In [ ]:
!pip install xgboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # data visualization
import seaborn as sns # statistical data visualization
import random


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.ensemble import AdaBoostClassifier

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader


from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
import lightgbm as lgb
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import accuracy_score

# Any results you write to the current directory are saved as output.
# ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [ ]:
CFG = {
    'IMG_SIZE':512,
    'EPOCHS':5,
    'LEARNING_RATE':1e-4,
    'BATCH_SIZE':16,
    'SEED':41
}

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/test_cancer_cls.csv')
train_df.loc[66,'PR_Allred_score'] = np.NaN
train_df.loc[67,'PR_Allred_score'] = np.NaN

target_name = ['N_category']

# 범주형 변수
cat_col_names = ['진단명','암의 위치', '암의 개수', 'NG','HG','HG_score_1','HG_score_2','HG_score_3','DCIS_or_LCIS_여부','DCIS_or_LCIS_type','T_category','ER','PR','HER2','HER2_IHC','HER2_SISH','BRCA_mutation']

#수치형 
num_col_names = ['나이','암의 장경','ER_Allred_score','PR_Allred_score','KI-67_LI_percent','HER2_SISH_ratio']


train_df['암의 장경'] = train_df['암의 장경'].fillna(train_df['암의 장경'].median())
train_df = train_df.fillna(0)

test_df['암의 장경'] = test_df['암의 장경'].fillna(train_df['암의 장경'].median())
test_df = test_df.fillna(0)



train_df = train_df.drop(['ID', 'img_path', 'mask_path', '수술연월일'], axis =1)
test_df = test_df.drop(['ID', 'img_path', '수술연월일'], axis = 1)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
                                                    train_df.drop(columns=['N_category']), 
                                                    train_df['N_category'], 
                                                    test_size=0.1, 
                                                    random_state=CFG['SEED'])

In [ ]:
def get_values(value):
    return value.values.reshape(-1, 1)

numeric_cols = ['나이', '암의 장경', 'ER_Allred_score', 'PR_Allred_score', 'KI-67_LI_percent', 'HER2_SISH_ratio']
ignore_cols = ['ID', 'img_path', 'mask_path', '수술연월일', 'N_category']

for col in train_df.columns:
    if col in ignore_cols:
        continue
    if col in numeric_cols:
        scaler = StandardScaler()
        X_train[col] = scaler.fit_transform(get_values(X_train[col]))
        X_test[col] = scaler.transform(get_values(X_test[col]))
        test_df[col] = scaler.transform(get_values(test_df[col]))
    else:
        le = LabelEncoder()
        X_train[col] = le.fit_transform(get_values(X_train[col]))
        X_test[col] = le.transform(get_values(X_test[col]))
        test_df[col] = le.transform(get_values(test_df[col]))

In [ ]:
clf = XGBClassifier() # 모델 선언

clf.fit(X_train,y_train) # 모델 학습

XGBClassifier()

In [ ]:
#mean 0
y_pred=clf.predict(X_test) # 예측 라벨

print("f1_score 각 분할의 정확도 기록 :", f1_score(y_pred, y_test))

f1_score 각 분할의 정확도 기록 : 0.8000000000000002


In [ ]:
#mean 0
y_pred=clf.predict(X_train) # 예측 라벨

print("f1_score 각 분할의 정확도 기록 :", f1_score(y_pred, y_train))

f1_score 각 분할의 정확도 기록 : 0.8762322015334063


In [ ]:
y_pred=clf.predict(test_df) # 예측 라벨

print(y_pred)

[1 0 0 0 1 1 1 1 1 1 0 1 1 0 0 0 1 0 1 1 1 1 0 1 0 1 1 0 1 0 1 0 1 1 1 1 0
 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 0 1 1 0 1 0 1 0 0 0 1 0 1 0 0 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 0 1
 1 1 0 0 1 0 0 1 1 0 1 1 1 1 1 1 0 0 1 0 1 0 0 1 0 0 0 0 1 0 0 1 0 0 1 1 0
 0 1 1 1 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 1 1 0 1 0 1 0 1 0 0 1 0 1 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0 1 0 0 0 1 0 1
 1 0 1 0 1 0 0 0 0 1 1 0 1 0 1 0 1 1 0 1 0 0 0 1 0 0 0 0]


In [ ]:
y_pred=clf.predict(test_df) # 예측 라벨

print(y_pred)

[1 0 0 0 1 1 1 0 1 1 1 1 1 0 0 1 1 0 1 0 1 1 0 0 0 1 1 0 1 0 1 0 1 1 1 1 0
 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 0 1 1 0 1 0 1 0 0 0 1 0 1 0 0 0 1 0 1 1 1 1 1 1 1 1 1 1 1 0 0 0 1 0 1
 1 1 0 0 1 0 0 1 1 1 1 1 1 1 1 1 0 0 1 0 1 0 1 1 0 0 1 0 0 0 0 1 0 0 1 1 0
 0 1 1 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 0 0 1 0 1 0 0 0 0 1 1 1 0 0 0 1
 0 1 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 1
 1 0 1 0 1 0 0 0 0 1 1 0 1 0 1 0 1 1 1 1 0 0 0 1 1 0 0 0]


In [ ]:
submit = pd.read_csv('/content/drive/MyDrive/sample_submission_cancer_cls.csv')
submit['N_category'] = y_pred
submit.to_csv('./XGBoost_submit_median.csv', index=False)

In [ ]:
clf = CatBoostClassifier() # 모델 선언

clf.fit(X_train,y_train) # 모델 학습

Learning rate set to 0.009849
0:	learn: 0.6888185	total: 1.63ms	remaining: 1.63s
1:	learn: 0.6841352	total: 3.42ms	remaining: 1.71s
2:	learn: 0.6796271	total: 5.3ms	remaining: 1.76s
3:	learn: 0.6755526	total: 7.1ms	remaining: 1.77s
4:	learn: 0.6706491	total: 8.88ms	remaining: 1.77s
5:	learn: 0.6660234	total: 10.5ms	remaining: 1.75s
6:	learn: 0.6629028	total: 11.8ms	remaining: 1.67s
7:	learn: 0.6585016	total: 13.4ms	remaining: 1.66s
8:	learn: 0.6538380	total: 15.2ms	remaining: 1.68s
9:	learn: 0.6495908	total: 16.9ms	remaining: 1.67s
10:	learn: 0.6459506	total: 18.5ms	remaining: 1.67s
11:	learn: 0.6415129	total: 20.3ms	remaining: 1.67s
12:	learn: 0.6382938	total: 22ms	remaining: 1.67s
13:	learn: 0.6344858	total: 23.8ms	remaining: 1.68s
14:	learn: 0.6307742	total: 25.6ms	remaining: 1.68s
15:	learn: 0.6275921	total: 27.9ms	remaining: 1.72s
16:	learn: 0.6238833	total: 29.9ms	remaining: 1.73s
17:	learn: 0.6211315	total: 31.5ms	remaining: 1.72s
18:	learn: 0.6178606	total: 33.3ms	remaining: 1.

In [ ]:
#mean 0
y_pred=clf.predict(X_test) # 예측 라벨

print("f1_score 각 분할의 정확도 기록 :", f1_score(y_pred, y_test))

f1_score 각 분할의 정확도 기록 : 0.8113207547169812


In [ ]:
#mean 0
y_pred=clf.predict(X_train) # 예측 라벨

print("f1_score 각 분할의 정확도 기록 :", f1_score(y_pred, y_train))

f1_score 각 분할의 정확도 기록 : 0.9473684210526316


In [ ]:
y_pred=clf.predict(test_df) # 예측 라벨

print(y_pred)

[1 1 0 0 1 1 1 1 1 1 0 1 1 0 0 1 1 0 1 0 1 1 0 1 0 1 1 0 1 0 1 0 1 1 1 1 0
 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 0 1 0 1 0 0 0 1 0 1 0 0 0 1 0 1 1 1 1 1 1 1 1 1 0 0 1 0 0 1 0 1
 1 1 0 0 1 0 0 1 1 0 1 1 1 1 1 1 0 0 1 1 1 0 0 1 0 0 0 0 0 0 0 1 0 0 1 1 1
 0 0 1 1 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 1 0 1 0 1 0 0 0 1 0 0 1 0 1 0 0 0 1
 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 1 1 0 1 0 0 0 0 0 1
 0 0 1 0 1 0 0 0 0 0 1 0 1 1 1 0 1 0 1 1 0 0 0 1 0 0 0 0]


In [ ]:
submit = pd.read_csv('/content/drive/MyDrive/sample_submission_cancer_cls.csv')
submit['N_category'] = y_pred
submit.to_csv('./CatBoost_submit_median.csv', index=False)

In [ ]:
clf = GradientBoostingClassifier() # 모델 선언

clf.fit(X_train,y_train) # 모델 학습

In [ ]:
#mean 0
y_pred=clf.predict(X_test) # 예측 라벨

print("f1_score 각 분할의 정확도 기록 :", f1_score(y_pred, y_test))

In [ ]:
#mean 0
y_pred=clf.predict(X_train) # 예측 라벨

print("f1_score 각 분할의 정확도 기록 :", f1_score(y_pred, y_train))

In [ ]:
y_pred=clf.predict(test_df) # 예측 라벨

print(y_pred)

In [ ]:
submit = pd.read_csv('/content/drive/MyDrive/sample_submission_cancer_cls.csv')
submit['N_category'] = y_pred
submit.to_csv('./GradientBoosting_submit.csv', index=False)

In [ ]:
clf = CatBoostClassifier() # 모델 선언

clf.fit(X_train,y_train) # 모델 학습

In [ ]:
#mean 0
y_pred=clf.predict(X_test) # 예측 라벨

print("f1_score 각 분할의 정확도 기록 :", f1_score(y_pred, y_test))

In [ ]:
#mean 0
y_pred=clf.predict(X_train) # 예측 라벨

print("f1_score 각 분할의 정확도 기록 :", f1_score(y_pred, y_train))

In [ ]:
y_pred=clf.predict(test_df) # 예측 라벨

print(y_pred)

In [ ]:
submit = pd.read_csv('/content/drive/MyDrive/sample_submission_cancer_cls.csv')
submit['N_category'] = y_pred
submit.to_csv('./CatBoost_submit.csv', index=False)

In [ ]:
clf =lgb.LGBMClassifier()
clf.fit(X_train,y_train) # 모델 학습

In [ ]:
#mean 0
y_pred=clf.predict(X_test) # 예측 라벨

print("f1_score 각 분할의 정확도 기록 :", f1_score(y_pred, y_test))

In [ ]:
#mean 0
y_pred=clf.predict(X_train) # 예측 라벨

print("f1_score 각 분할의 정확도 기록 :", f1_score(y_pred, y_train))

In [ ]:
y_pred=clf.predict(test_df) # 예측 라벨

print(y_pred)

In [ ]:
submit = pd.read_csv('/content/drive/MyDrive/sample_submission_cancer_cls.csv')
submit['N_category'] = y_pred
submit.to_csv('./LGBMClassifier.csv', index=False)

In [ ]:
clf = AdaBoostClassifier() # 모델 선언

clf.fit(X_train,y_train) # 모델 학습

AdaBoostClassifier()

In [ ]:
#mean 0
y_pred=clf.predict(X_test) # 예측 라벨

print("f1_score 각 분할의 정확도 기록 :", f1_score(y_pred, y_test))

f1_score 각 분할의 정확도 기록 : 0.8000000000000002


In [ ]:
#mean 0
y_pred=clf.predict(X_train) # 예측 라벨

print("f1_score 각 분할의 정확도 기록 :", f1_score(y_pred, y_train))

f1_score 각 분할의 정확도 기록 : 0.8016877637130801


In [ ]:
y_pred=clf.predict(test_df) # 예측 라벨

print(y_pred)

[1 0 0 0 1 1 1 1 1 1 1 0 1 1 0 1 1 0 1 0 1 1 0 1 0 1 1 0 1 1 1 0 1 1 1 1 0
 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 0 1 1 0 1 0 1 0 0 0 1 0 1 0 0 0 1 0 1 1 1 1 1 1 1 1 1 0 0 1 0 0 1 0 1
 1 1 0 0 1 0 0 1 1 1 1 0 1 1 1 1 0 1 1 1 1 0 0 1 0 0 0 0 1 0 1 1 0 1 1 0 0
 0 1 1 1 0 0 0 0 0 0 1 1 1 1 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 1 0 0 0 1
 0 0 0 0 0 0 1 1 1 1 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 1 0 1 1 0 1 0 0 0 0 0 1
 1 0 1 0 1 1 0 1 0 1 1 0 1 0 0 0 1 1 1 1 0 0 0 1 0 0 0 0]


In [ ]:
submit = pd.read_csv('/content/drive/MyDrive/sample_submission_cancer_cls.csv')
submit['N_category'] = y_pred
submit.to_csv('./AdaBoost_submit_(1_9train_test_split).csv', index=False)

In [ ]:
clf = ExtraTreesClassifier() # 모델 선언

clf.fit(X_train,y_train) # 모델 학습

In [ ]:
#mean 0
y_pred=clf.predict(X_test) # 예측 라벨

print("f1_score 각 분할의 정확도 기록 :", f1_score(y_pred, y_test))

In [ ]:
#mean 0
y_pred=clf.predict(X_train) # 예측 라벨

print("f1_score 각 분할의 정확도 기록 :", f1_score(y_pred, y_train))

In [ ]:
y_pred=clf.predict(test_df) # 예측 라벨

print(y_pred)

In [ ]:
submit = pd.read_csv('/content/drive/MyDrive/sample_submission_cancer_cls.csv')
submit['N_category'] = y_pred
submit.to_csv('./ExtraTreesClassifier_submit.csv', index=False)

In [ ]:
clf = RidgeClassifier() # 모델 선언

clf.fit(X_train,y_train) # 모델 학습

RidgeClassifier()

In [ ]:
#mean 0
y_pred=clf.predict(X_test) # 예측 라벨

print("f1_score 각 분할의 정확도 기록 :", f1_score(y_pred, y_test))

f1_score 각 분할의 정확도 기록 : 0.7735849056603773


In [ ]:
#mean 0
y_pred=clf.predict(X_train) # 예측 라벨

print("f1_score 각 분할의 정확도 기록 :", f1_score(y_pred, y_train))

f1_score 각 분할의 정확도 기록 : 0.7826086956521741


In [ ]:
y_pred=clf.predict(test_df) # 예측 라벨

print(y_pred)

[0 0 0 0 1 1 1 1 0 1 1 0 1 1 0 1 1 0 1 0 1 1 0 1 0 1 1 0 1 1 1 0 1 1 1 1 0
 0 1 0 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 0 1 1 0 1 0 1 0 0 0 1 1 1 0 0 0 1 0 1 1 1 1 1 1 1 1 1 0 0 1 0 0 1 0 1
 1 1 0 0 1 0 0 1 1 0 1 0 1 1 0 1 0 1 1 1 1 0 0 1 0 0 1 0 0 0 0 1 0 0 1 1 0
 1 0 1 1 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0 1 1 0 0 1 0 1 0 0 0 1 1 1 1 0 0 0 1
 0 0 0 0 0 0 1 1 1 1 1 0 1 0 0 0 0 0 0 1 0 0 0 0 1 1 0 0 1 0 1 0 0 0 1 0 1
 1 0 0 0 1 1 0 0 0 1 0 0 0 0 1 0 1 1 0 0 0 0 0 0 0 0 0 0]


In [ ]:
submit = pd.read_csv('/content/drive/MyDrive/sample_submission_cancer_cls.csv')
submit['N_category'] = y_pred
submit.to_csv('./RidgeClassifier_submit.csv', index=False)